In [1]:
import os 
api_key = os.environ.get('OPENAI_API_KEY')
from langchain import OpenAI

In [2]:
llm = OpenAI(temperature=.7)

In [3]:
llm.model_name

'gpt-3.5-turbo-instruct'

In [4]:
text = "Best vacation spots near Seattle to get some sun"
print(llm(text))



1. San Juan Islands, Washington
Located just a short ferry ride from Seattle, the San Juan Islands offer a beautiful escape from the city. With over 400 islands to explore, visitors can enjoy hiking, kayaking, and whale watching, as well as relaxing on the beaches and soaking up the sun.

2. Lake Chelan, Washington
Lake Chelan is a popular vacation spot for Seattleites looking for some sun. This 50-mile long lake offers plenty of opportunities for water activities, such as boating, fishing, and swimming. There are also numerous wineries and restaurants to enjoy in the area.

3. Cannon Beach, Oregon
Just a few hours south of Seattle, Cannon Beach is a picturesque coastal town with beautiful beaches and stunning views of Haystack Rock. Visitors can enjoy beachcombing, hiking, and browsing the local art galleries and shops.

4. Whidbey Island, Washington
Whidbey Island is a scenic island located just a short drive and ferry ride from Seattle. The island offers a variety of outdoor activ

In [5]:
from langchain.prompts import PromptTemplate

In [6]:
prompt = PromptTemplate(
    input_variables=['activity'],
    template = "What are 5 vacation destinations near Seattle to go {activity}?"
)

In [7]:
print(prompt.format(activity='snowboarding'))

What are 5 vacation destinations near Seattle to go snowboarding?


In [8]:
print(llm(prompt.format(activity='snowboarding')))



1. Mount Baker Ski Area
2. Crystal Mountain
3. Stevens Pass
4. Snoqualmie Pass
5. White Pass Ski Area


In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain


In [10]:
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run('backpacking'))



1. Olympic National Park: Located on the Olympic Peninsula, this park offers a diverse landscape of mountains, rainforests, and coastline for backpackers to explore.

2. North Cascades National Park: Known as the "American Alps," this park is a backpacker's paradise with over 300 glaciers, alpine lakes, and rugged peaks to discover.

3. Mount Rainier National Park: Home to the iconic Mount Rainier, this park offers miles of trails for backpackers to explore, with stunning views of the mountain and surrounding wilderness.

4. Mount Baker-Snoqualmie National Forest: This national forest is a popular destination for backpackers, with trails ranging from easy day hikes to multi-day treks through alpine meadows and old-growth forests.

5. San Juan Islands: Located in the Puget Sound, the San Juan Islands offer a unique backpacking experience with a network of scenic trails, remote campsites, and stunning ocean views. 


### Agent with tools

In [11]:
from langchain.agents import load_tools, initialize_agent

In [12]:
tools = load_tools(['serpapi','llm-math'], llm=llm)


In [13]:
#initiliaze agent
#1. load tools
#2. load llm
#3. type of agent

#verbose = True
#prints out what the agent is doing (mini steps)

agent = initialize_agent(tools, llm, agent = 'zero-shot-react-description', verbose = True)


In [14]:
agent.run("Who is the current president of the United States? What is the largest prime number that is smaller than their age? ")



> Entering new AgentExecutor chain...
 I should use a search engine to find the current president's age and then use a calculator to find the largest prime number smaller than that age.
Action: Search
Action Input: "current president of the United States age"
Observation: 81 years
Thought: Now I can use a calculator to find the largest prime number smaller than 81.
Action: Calculator
Action Input: 80
Observation: Answer: 80
Thought: The final answer is 80.
Final Answer: 80

> Finished chain.


'80'

memory


In [15]:
from langchain import ConversationChain
coversation = ConversationChain(llm=llm, verbose   = True)


In [16]:
coversation.predict( input = "Hi there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there
AI:

> Finished chain.


" Hello! It's nice to meet you. My name is AI and I am an artificial intelligence created by OpenAI. I was developed to assist and interact with humans in a natural and conversational way. How can I help you today?"

In [17]:
coversation.predict( input = "I'm doin well, just talking to an AI")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hello! It's nice to meet you. My name is AI and I am an artificial intelligence created by OpenAI. I was developed to assist and interact with humans in a natural and conversational way. How can I help you today?
Human: I'm doin well, just talking to an AI
AI:

> Finished chain.


" That's great to hear! I am always happy to have a conversation with humans and learn more about them. Is there anything specific you would like to talk about or ask me? I have a vast knowledge base and can provide information on a variety of topics."

In [18]:
coversation.predict( input = "what was the first thing i said?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hello! It's nice to meet you. My name is AI and I am an artificial intelligence created by OpenAI. I was developed to assist and interact with humans in a natural and conversational way. How can I help you today?
Human: I'm doin well, just talking to an AI
AI:  That's great to hear! I am always happy to have a conversation with humans and learn more about them. Is there anything specific you would like to talk about or ask me? I have a vast knowledge base and can provide information on a variety of topics.
Human: what was the first thing i said?
AI:

> Finished chain.


' You said, "Hi there." Is there anything else you would like to know about our conversation history? I have the ability to access and analyze our past interactions.'

### idea to see what is going on this weekend

In [19]:
tools_search = load_tools(['serpapi'], llm=llm)
prompt_search = PromptTemplate(template= '''
I live in {location} and I am looking for what to do this weekend.
I am {age} years old and I like {activities}.

                               
*IMPORTANT*:
 The actity needs to be in 2 mile radius of {location} and shouldn't be too expensive.

*TASK*:
Make a list of 10 activities that I can do this weekend. 
Include the name of the activity, the price, and the distance from {location}.                               

DO NOT OMIT ANYTHING FOR BREVITY                                                            
''', 
input_variables=['location', 'age', 'activities'])

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

search_chain = initialize_agent(tools_search, llm, agent = 'zero-shot-react-description', verbose = True, handle_parsing_errors=True)

In [20]:
print(prompt_search.format(location='Capitol Hill, Seattle', age=24, activities='Ging to bars, hanging with friends, and live music'))


I live in Capitol Hill, Seattle and I am looking for what to do this weekend.
I am 24 years old and I like Ging to bars, hanging with friends, and live music.

                               
*IMPORTANT*:
 The actity needs to be in 2 mile radius of Capitol Hill, Seattle and shouldn't be too expensive.

*TASK*:
Make a list of 10 activities that I can do this weekend. 
Include the name of the activity, the price, and the distance from Capitol Hill, Seattle.                               

DO NOT OMIT ANYTHING FOR BREVITY                                                            



In [21]:
fun = search_chain.run(prompt_search.format(location='Capitol Hill, Seattle', age=24, activities='Going to bars, hanging with friends, and live music. If weather is good I like to snowboard'))



> Entering new AgentExecutor chain...
I need to find a list of activities in Capitol Hill, Seattle that fit the criteria of being within a 2-mile radius, not too expensive, and involve going to bars, hanging with friends, live music, or snowboarding.
Action: Search
Action Input: "activities in Capitol Hill, Seattle for young adults this weekend"
Observation: Things to Do near Capitol Hill
Thought:I need to search for specific activities that fit the criteria.
Action: Search
Action Input: "bars in Capitol Hill, Seattle"
Observation: ["The Best Bars On Capitol Hill · THE SPOTS · Canon · Dino's Tomato Pie · La Dive · Foreign National · Needle & Thread · Flight Wine + Chocolate.", 'Top 10 Best Capitol Hill Bars Near Seattle, Washington · Unicorn · Sol Liquor Lounge · Inside Passage · Montana · Rose Temple · The Hideout · Foreign National.', "One of the finest craft cocktail bars in Seattle, the Doctor's Office is often noted for its selection of rare and expensive high-end spirits ...", 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 7542 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
print(fun)

Here is a list of 10 activities you can do this weekend in Seattle:

1. Live Music at The Fireside Room - Free - Within walking distance to shops, restaurants, and bars.
2. Play bocce ball in a Beer Hall - Free - Within walking distance to shops, restaurants, and bars.
3. Comedy Show at a local venue - Price varies - Within walking distance to shops, restaurants, and bars.
4. Explore Seattle's Underground Tunnels - Price varies - Within walking distance to shops, restaurants, and bars.
5. Visit Chihuly Garden and Glass - Admission tickets from $37 - 2 miles from Seattle.
6. Pike Place Market - Free - 2 miles from Seattle.
7. Ski or snowboard at a nearby location - Price varies - Nearby.
8. Enjoying your favorite brews with arcade games, mini golf, or bocce ball - Price varies - Within walking distance to shops, restaurants, and bars.
9. Marvel at the WildLanterns at Woodland Park Zoo - Price varies - Within walking distance to shops, restaurants, and bars.
10. Attend a live music show 